# Eigenvektoren an Rahmentragwerk mit Steifigkeitsmatrix {#sec-mms_rahmen}

[Aufgabenstellung](calculations/%C3%9Cbung%201_2_MMS.pdf)

![Statisches System des Rahmentragwerks](pictures/system_mms2.jpg){#fig-system}

In [1]:
import sympy as sp 
from sympycalcs import render, convert
import sympy.physics.units as unit

import matplotlib.pyplot as plt

## Parameter der Aufgabenstellung

In [2]:
params = {'E': 30*10**3 *unit.N/unit.mm**2,
          'I':2*10**9*unit.mm**4,
          'm_1':2*20000*unit.N*unit.second**2/unit.m,
          'm_2':20000*unit.N*unit.second**2/unit.m,
          'l':4*unit.m,
          'H':3.2*unit.m,
          
          }

render.dict_render(params)

Eq(E, 30000*newton/millimeter**2)

Eq(I, 2000000000*millimeter**4)

Eq(m_1, 40000*newton*second**2/meter)

Eq(m_2, 20000*newton*second**2/meter)

Eq(l, 4*meter)

Eq(H, 3.2*meter)

In [3]:
omega, t,  l, m_1, m_2, E,I, H = sp.symbols('omega, t,  l, m_1, m_2, E, I, H')

omega_n = sp.symbols('omega_n')

## Steifigkeitsmatrix $\mathbf{K}$

Zur Bestimmung der Steifigkeitsmatrix ist das System an jedem Freiheitsgrad auszulenken, wie in @fig-steifigkeit dargestellt ist.

![Auslenkung der Freiheitsgrade zur Bestimmung der Steifigkeit](pictures/steifigkeit_mms2.jpg){#fig-steifigkeit}

Wichtig dabei sind die Richtungen der Kräfte. Als Denkstütze gilt folgendes:
- Der Auslenkung um $u$ wirkt die Federkraft entgegen, welche $k u$ entspricht.
- Zusätzlich wirkt die Trägheitskraft der Auslenkung entgegen, welche $m u''$ entspricht.
- Nach der Betrachtung des ausgelenkten Punkts, kann mittels *Actio-Reactio*-Prinzip das "*Stockwerk*" ins Gleichgewicht gebracht werden.
- Vorzeichen sind gegen der Bewegungsrichtig positiv.

### Horizontale Steifigkeit

Die horizontale Steifigkeit des Systems lässt sich mittels der Formel für Rahmen bestimmen.

$$k_{Stütze} = \frac{12EI_{Stütze}}{H^3}$$

Diese gilt für eine einzelne Stütze.

In [4]:
k_1, k_2 = sp.symbols('k_1, k_2')

params['k_1'] = 2*(12 * 2*E*I / H**3).subs(params).simplify().evalf(6)
params['k_2'] = 2*(12 * E*I / H**3).subs(params).simplify().evalf(6)
K = sp.Matrix([[k_1 + k_2, -k_2],[-k_2, k_2]])


render.eq_display(sp.MatrixSymbol('K', 2,2), K,
                  sp.MatrixSymbol('K', 2,2), K.subs(params),
                  )

Eq(K, Matrix([
[k_1 + k_2, -k_2],
[     -k_2,  k_2]]))

Eq(K, Matrix([
[ 1.31836e+8*newton/meter, -4.39453e+7*newton/meter],
[-4.39453e+7*newton/meter,  4.39453e+7*newton/meter]]))

## Eigenvektoren
### Massenmatrix $\mathbf{M}$ 

Die Massenmatrix folgt dem gleichen Aufbau wie die Steifigkeitsmatrix. Es gelten die gleichen Vorzeichenregelungen.

In [5]:
m_1, m_2 = sp.symbols('m_1, m_2')
M = sp.Matrix([[m_1, 0],[0, m_2]])

render.eq_display(sp.MatrixSymbol('M', 2,2), M,
                  sp.MatrixSymbol('M', 2,2), M.subs(params))

Eq(M, Matrix([
[m_1,   0],
[  0, m_2]]))

Eq(M, Matrix([
[40000*newton*second**2/meter,                            0],
[                           0, 20000*newton*second**2/meter]]))

### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand folgender Gleichung bestimmen:

$$\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}$$

In [6]:
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

Eq(omega_1, 33.1/second)

Eq(omega_2, 66.3/second)

### Eigenvektoren $\phi$

In [7]:
phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')
params['phi_21'] = 1

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_11 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_11).values())[0]

params['phi_11'] = phi_11.subs(params).simplify() 




render.eq_display((K-omega_1**2 *M)*phi_1,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))

Eq(Matrix([
[-k_2*phi_21 + phi_11*(k_1 + k_2 - m_1*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) - sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))],
[      -k_2*phi_11 + phi_21*(k_2 - m_2*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) - sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))]]), Matrix([
[0],
[0]]))

Eq(phi_1, Matrix([
[0.5],
[1.0]]))

In [8]:
params['phi_22'] = 1

phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_12 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_12).values())[0]

params['phi_12'] = phi_12.subs(params).simplify() 




render.eq_display((K-omega_2**2 *M)*phi_2,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))

Eq(Matrix([
[-k_2*phi_22 + phi_12*(k_1 + k_2 - m_1*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) + sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))],
[      -k_2*phi_12 + phi_22*(k_2 - m_2*(k_1/(2*m_1) + k_2/(2*m_2) + k_2/(2*m_1) + sqrt(k_1**2*m_2**2 - 2*k_1*k_2*m_1*m_2 + 2*k_1*k_2*m_2**2 + k_2**2*m_1**2 + 2*k_2**2*m_1*m_2 + k_2**2*m_2**2)/(2*m_1*m_2)))]]), Matrix([
[0],
[0]]))

Eq(phi_2, Matrix([
[-1.0],
[ 1.0]]))

### Orthogonalitätsbedingung

Zur effektiven Entkoppelung der Gleichungen muss die Orthogonalitätsbedingung eingehalten sein. Dies gilt es für die Massenmatrix zu kontrollieren:

$$\mathbf{\phi_1^T M \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_1 = 0}$$

Sowohl auch für die Steifigkeitsmatrix:

$$\mathbf{\phi_1^T K \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_1 = 0}$$



In [9]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_2.T*M*phi_1).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(3))

Eq(phi_1.T*M*phi_1, Matrix([[3.0e+4*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_2, Matrix([[6.0e+4*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_1, Matrix([[0]]))

Eq(phi_1.T*M*phi_2, Matrix([[0]]))

In [10]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(4),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(4))

Eq(phi_1.T*K*phi_1, Matrix([[3.3e+7*newton/meter]]))

Eq(phi_2.T*K*phi_2, Matrix([[2.64e+8*newton/meter]]))

Eq(phi_2.T*K*phi_1, Matrix([[0]]))

Eq(phi_1.T*K*phi_2, Matrix([[0]]))

### Eigenformen

![Die beiden Eigenformen skizziert](pictures/eigenformen_mms2.jpg){#fig-eigenformen}